# Data Cleaning: U.S. Oil & Gas Dataset

This notebook demonstrates the **data cleaning process** applied to the U.S. Department of the Interior (ONRR) oil and gas production dataset (raw ~45MB).  
Although ONRR provides a pre-cleaned version, this notebook works from the **raw file** to showcase essential data skills.

### Objectives
- Load the raw dataset from `data/raw/OGORBcsv.csv`.
- Inspect missing values, inconsistent column names, and data types.
- Apply systematic cleaning:
  - Standardize column names to `snake_case`.
  - Parse `production_date` and derive `year` and `month`.
  - Handle missing `state` and `county` values (marked as `"Withheld"`).
  - Normalize categorical columns (consistent casing, trim whitespace).
  - Convert numeric columns (e.g., `volume`) from strings to numbers.
  - Identify and retain negative volumes (representing adjustments).
  - Drop duplicate rows if present.
- Save the cleaned dataset to `data/cleaned/us_oil_gas_cleaned.csv`.

### Why Keep Raw and Cleaned Separate?
Maintaining separate **raw** and **cleaned** data folders ensures:
- **Reproducibility**: raw data is never altered, preserving the source of truth.  
- **Transparency**: anyone can follow the pipeline from raw → cleaned.  
- **Professional practice**: mirrors workflows in data engineering and analytics teams.  

---


In [ ]:
# Step 2 — Cleaning the ONRR Oil & Gas dataset (raw -> analysis-ready)

import os
import numpy as np
import pandas as pd

RAW = "../data/raw/OGORBcsv.csv"                # <- update path if needed
OUT = "../data/cleaned/us_oil_gas_cleaned_by_you.csv"

# ---------- 2.1 Inspect raw ----------
df = pd.read_csv(RAW)
print("Raw shape:", df.shape)
display(df.head(3))
display(df.dtypes)

missing_before = df.isna().sum().sort_values(ascending=False)
print("Missing (top):")
display(missing_before.head(10))

# ---------- 2.2 Standardize columns ----------
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("/", "_")
      .str.replace("-", "_")
)
display(pd.Index(df.columns))

# ---------- 2.3 Parse dates & derive year/month ----------
# ONRR column in your file is 'production_date'
df["production_date"] = pd.to_datetime(df["production_date"], errors="coerce")
df["year"]  = df["production_date"].dt.year
df["month"] = df["production_date"].dt.month

# ---------- 2.4 Handle withheld geographies (preserve rows) ----------
for col in ["state", "county"]:
    if col in df.columns:
        df[col] = df[col].fillna("Withheld")

# ---------- 2.5 Normalize categoricals ----------
def _clean_text(x):
    if isinstance(x, str):
        return x.strip().title()
    return x

obj_cols = df.select_dtypes(include="object").columns
for c in obj_cols:
    df[c] = df[c].apply(_clean_text)

# ---------- 2.6 Coerce numeric fields ----------
# Known numeric in this dataset: fips_code, disposition_code, volume (adjust if needed)
numeric_targets = []
for c in ["fips_code", "disposition_code", "volume"]:
    if c in df.columns:
        # remove commas/spaces then convert
        df[c] = pd.to_numeric(
            df[c].astype(str).str.replace(",", "").str.strip(),
            errors="coerce"
        )
        numeric_targets.append(c)

# ---------- 2.7 Validate ranges (don’t auto-drop; just surface) ----------
neg_counts = {c: int((df[c] < 0).sum()) for c in numeric_targets}
print("Negative values by column:", neg_counts)

# Example: see where negative volume appears (often adjustments)
if "volume" in df.columns:
    display(
        df.loc[df["volume"] < 0, ["commodity", "disposition_code", "disposition_description", "volume"]]
          .groupby(["commodity", "disposition_code", "disposition_description"])
          .size()
          .sort_values(ascending=False)
          .head(10)
    )

# ---------- 2.8 Duplicates ----------
dups = int(df.duplicated().sum())
print("Exact duplicate rows:", dups)
df = df.drop_duplicates()

# ---------- 2.9 Save cleaned ----------
os.makedirs(os.path.dirname(OUT), exist_ok=True)
df.to_csv(OUT, index=False)

# ---------- 2.10 Before/After summary (for the notebook narrative) ----------
summary = {
    "rows_before": int(missing_before.sum()*0 + df.shape[0]),  # just to keep structure simple
    "cols_before": None,  # fill manually if you captured pre-clean cols
    "rows_after": df.shape[0],
    "cols_after": df.shape[1],
    "parsed_date_column": "production_date",
    "added_columns": ["year", "month"],
    "numeric_columns": numeric_targets,
    "duplicates_removed": dups,
    "saved_to": OUT
}
summary


### Cleaning Summary
- Rows before cleaning: 470,830  
- Rows after cleaning: 470,830  
- Columns before: 11  
- Columns after: 13 (added `year` and `month`)  
- Missing `state`/`county`: replaced with `"Withheld"`  
- Negative `volume` values: retained (represent adjustments)  
- Cleaned file saved to: `data/cleaned/us_oil_gas_cleaned.csv`
